Imported required libraries: OS, Numpy, Keras, CV2, dlib

In [1]:
import os
#import numpy as np
#from keras.preprocessing import image
import cv2
#import dlib
import csv

read the Image from a location.
TODO: Check the relative image path. Currently C: drive location linked.
Create img object: get image from the location, convert it into greyscale via img = cv.imread(url, cv2.IMREAD_GRAYSCALE)
TODO: check image objects

In [2]:
#global basedir, image_paths, target_size
basedir = '../Datasets/celeba'
images_dir = os.path.join(basedir,'img')
labels_filename = 'labels.csv'
correct_labels = dict()

In [3]:
with open('../Datasets/celeba/labels.csv', 'r') as file:
    reader = csv.reader(file, delimiter = '\t')
    next(reader)
    #reader = csv.DictReader(file, delimiter = '\t')
    for row in reader:
        correct_labels[row[1]] = row
    print(correct_labels)

{'0.jpg': ['0', '0.jpg', '-1', '1'], '1.jpg': ['1', '1.jpg', '-1', '1'], '2.jpg': ['2', '2.jpg', '1', '-1'], '3.jpg': ['3', '3.jpg', '-1', '-1'], '4.jpg': ['4', '4.jpg', '-1', '-1'], '5.jpg': ['5', '5.jpg', '-1', '-1'], '6.jpg': ['6', '6.jpg', '1', '-1'], '7.jpg': ['7', '7.jpg', '1', '-1'], '8.jpg': ['8', '8.jpg', '-1', '1'], '9.jpg': ['9', '9.jpg', '-1', '-1'], '10.jpg': ['10', '10.jpg', '-1', '1'], '11.jpg': ['11', '11.jpg', '1', '1'], '12.jpg': ['12', '12.jpg', '1', '1'], '13.jpg': ['13', '13.jpg', '-1', '1'], '14.jpg': ['14', '14.jpg', '1', '-1'], '15.jpg': ['15', '15.jpg', '1', '1'], '16.jpg': ['16', '16.jpg', '-1', '1'], '17.jpg': ['17', '17.jpg', '-1', '1'], '18.jpg': ['18', '18.jpg', '-1', '-1'], '19.jpg': ['19', '19.jpg', '1', '-1'], '20.jpg': ['20', '20.jpg', '1', '1'], '21.jpg': ['21', '21.jpg', '-1', '-1'], '22.jpg': ['22', '22.jpg', '1', '1'], '23.jpg': ['23', '23.jpg', '-1', '-1'], '24.jpg': ['24', '24.jpg', '1', '-1'], '25.jpg': ['25', '25.jpg', '-1', '-1'], '26.jpg': ['

Create a list for gender

In [4]:
gender_list = ['1', '-1']

Define a function to load caffemodel and prototxt of gender detection, which is a pre-trained CNN model that does the detection

In [5]:
#def load_caffe_models():
gender_net = cv2.dnn.readNetFromCaffe('deploy_gender.prototxt', 'gender_net.caffemodel')
MODEL_MEAN_VALUES = (78.4263377603, 87.7689143744, 114.895847746)


Load the pre-existing model for facial detection.

In [6]:
#face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
face_detectors = [ cv2.CascadeClassifier("haarcascade_frontalface_default.xml"),
                  cv2.CascadeClassifier("haarcascade_frontalface_alt2.xml"),
                  cv2.CascadeClassifier("haarcascade_frontalface_alt.xml"),
                  cv2.CascadeClassifier("haarcascade_frontalface_alt_tree.xml"), 
                  cv2.CascadeClassifier("haarcascade_profileface.xml") ]

Create a general function to detect faces using OpenCV's detectMultiScale(image, scaleFactor, minNeighbors). The 'img' is our read grayscale image; leave scalefactor at 1; minNeighbors defines how many objects are found near current one before it declares the face found; in our case there is only one face per image but in some images there are other objects (e.g. hair flowers), so it is set to 5. If the classifier finds a face, it returns a list of positions of this face in the form “Rect(x,y,w,h).”, if not, then returns “None”.

Loop through the list of faces, break them up, set the size, and draw rectangles around them. 
TODO: Check the accuracy calculation - is faces the right parameter to use? Add NONE for those that cannot be detected.

In [7]:
image_filelist = os.listdir(images_dir)
counter=0
predicted_labels = list()
print(len(predicted_labels))
print(len(image_filelist))
for file in image_filelist:
    if file.lower().endswith('.jpg'):
        print(f'processing file %s', file)
        image_path = os.path.join(images_dir, file)
        img = cv2.imread(image_path,1)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        for face_detector in face_detectors: 
            faces = face_detector.detectMultiScale(gray, 1.3, 5)
            if len(faces) >0:
                break
        for (x,y,w,h) in faces:
            cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
        #    roi_gray = gray[y:y+h, x:x+w]
        #    roi_color = img[y:y+h, x:x+w]
        #cv2.waitKey(0)
        #cv2.destroyAllWindows()
        #Get Face
        face_img = img[y:y+h, h:h+w].copy()
        blob = cv2.dnn.blobFromImage(face_img, 1, (227, 227), MODEL_MEAN_VALUES, swapRB=False)
        #Predict Gender
        gender_net.setInput(blob)
        gender_preds = gender_net.forward()
        gender = gender_list[gender_preds[0].argmax()]
        predicted_labels.append((file, gender))
        counter += 1
        print(gender)
print(predicted_labels)
print(len(predicted_labels))
print(counter)

0
5000
processing file %s 0.jpg
-1
processing file %s 1.jpg
-1
processing file %s 10.jpg
-1
processing file %s 100.jpg
1
processing file %s 1000.jpg
-1
processing file %s 1001.jpg
-1
processing file %s 1002.jpg
1
processing file %s 1003.jpg
-1
processing file %s 1004.jpg
1
processing file %s 1005.jpg
-1
processing file %s 1006.jpg
1
processing file %s 1007.jpg
1
processing file %s 1008.jpg
-1
processing file %s 1009.jpg
-1
processing file %s 101.jpg
1
processing file %s 1010.jpg
-1
processing file %s 1011.jpg
-1
processing file %s 1012.jpg
1
processing file %s 1013.jpg
-1
processing file %s 1014.jpg
-1
processing file %s 1015.jpg
1
processing file %s 1016.jpg
-1
processing file %s 1017.jpg
-1
processing file %s 1018.jpg
1
processing file %s 1019.jpg
1
processing file %s 102.jpg
-1
processing file %s 1020.jpg
1
processing file %s 1021.jpg
1
processing file %s 1022.jpg
-1
processing file %s 1023.jpg
-1
processing file %s 1024.jpg
1
processing file %s 1025.jpg
-1
processing file %s 1026.j

1
processing file %s 1245.jpg
-1
processing file %s 1246.jpg
-1
processing file %s 1247.jpg
1
processing file %s 1248.jpg
-1
processing file %s 1249.jpg
-1
processing file %s 125.jpg
-1
processing file %s 1250.jpg
-1
processing file %s 1251.jpg
-1
processing file %s 1252.jpg
-1
processing file %s 1253.jpg
-1
processing file %s 1254.jpg
-1
processing file %s 1255.jpg
-1
processing file %s 1256.jpg
-1
processing file %s 1257.jpg
-1
processing file %s 1258.jpg
-1
processing file %s 1259.jpg
1
processing file %s 126.jpg
1
processing file %s 1260.jpg
-1
processing file %s 1261.jpg
-1
processing file %s 1262.jpg
-1
processing file %s 1263.jpg
-1
processing file %s 1264.jpg
-1
processing file %s 1265.jpg
-1
processing file %s 1266.jpg
-1
processing file %s 1267.jpg
-1
processing file %s 1268.jpg
-1
processing file %s 1269.jpg
-1
processing file %s 127.jpg
1
processing file %s 1270.jpg
-1
processing file %s 1271.jpg
-1
processing file %s 1272.jpg
-1
processing file %s 1273.jpg
-1
processing fi

processing file %s 1489.jpg
1
processing file %s 149.jpg
-1
processing file %s 1490.jpg
1
processing file %s 1491.jpg
-1
processing file %s 1492.jpg
-1
processing file %s 1493.jpg
-1
processing file %s 1494.jpg
-1
processing file %s 1495.jpg
-1
processing file %s 1496.jpg
-1
processing file %s 1497.jpg
-1
processing file %s 1498.jpg
-1
processing file %s 1499.jpg
-1
processing file %s 15.jpg
1
processing file %s 150.jpg
1
processing file %s 1500.jpg
-1
processing file %s 1501.jpg
1
processing file %s 1502.jpg
-1
processing file %s 1503.jpg
-1
processing file %s 1504.jpg
-1
processing file %s 1505.jpg
1
processing file %s 1506.jpg
-1
processing file %s 1507.jpg
-1
processing file %s 1508.jpg
-1
processing file %s 1509.jpg
1
processing file %s 151.jpg
-1
processing file %s 1510.jpg
-1
processing file %s 1511.jpg
1
processing file %s 1512.jpg
1
processing file %s 1513.jpg
1
processing file %s 1514.jpg
-1
processing file %s 1515.jpg
-1
processing file %s 1516.jpg
-1
processing file %s 1517

-1
processing file %s 1733.jpg
-1
processing file %s 1734.jpg
-1
processing file %s 1735.jpg
-1
processing file %s 1736.jpg
-1
processing file %s 1737.jpg
1
processing file %s 1738.jpg
-1
processing file %s 1739.jpg
-1
processing file %s 174.jpg
-1
processing file %s 1740.jpg
-1
processing file %s 1741.jpg
-1
processing file %s 1742.jpg
1
processing file %s 1743.jpg
-1
processing file %s 1744.jpg
-1
processing file %s 1745.jpg
-1
processing file %s 1746.jpg
-1
processing file %s 1747.jpg
1
processing file %s 1748.jpg
-1
processing file %s 1749.jpg
-1
processing file %s 175.jpg
1
processing file %s 1750.jpg
-1
processing file %s 1751.jpg
1
processing file %s 1752.jpg
-1
processing file %s 1753.jpg
-1
processing file %s 1754.jpg
1
processing file %s 1755.jpg
1
processing file %s 1756.jpg
-1
processing file %s 1757.jpg
1
processing file %s 1758.jpg
-1
processing file %s 1759.jpg
-1
processing file %s 176.jpg
-1
processing file %s 1760.jpg
-1
processing file %s 1761.jpg
1
processing file %

-1
processing file %s 1975.jpg
-1
processing file %s 1976.jpg
1
processing file %s 1977.jpg
1
processing file %s 1978.jpg
-1
processing file %s 1979.jpg
-1
processing file %s 198.jpg
1
processing file %s 1980.jpg
-1
processing file %s 1981.jpg
-1
processing file %s 1982.jpg
1
processing file %s 1983.jpg
-1
processing file %s 1984.jpg
-1
processing file %s 1985.jpg
-1
processing file %s 1986.jpg
-1
processing file %s 1987.jpg
1
processing file %s 1988.jpg
1
processing file %s 1989.jpg
-1
processing file %s 199.jpg
1
processing file %s 1990.jpg
-1
processing file %s 1991.jpg
1
processing file %s 1992.jpg
-1
processing file %s 1993.jpg
-1
processing file %s 1994.jpg
-1
processing file %s 1995.jpg
1
processing file %s 1996.jpg
-1
processing file %s 1997.jpg
-1
processing file %s 1998.jpg
-1
processing file %s 1999.jpg
-1
processing file %s 2.jpg
-1
processing file %s 20.jpg
-1
processing file %s 200.jpg
-1
processing file %s 2000.jpg
-1
processing file %s 2001.jpg
-1
processing file %s 200

...

-1
processing file %s 441.jpg
-1
processing file %s 4410.jpg
-1
processing file %s 4411.jpg
-1
processing file %s 4412.jpg
-1
processing file %s 4413.jpg
-1
processing file %s 4414.jpg
1
processing file %s 4415.jpg
1
processing file %s 4416.jpg
1
processing file %s 4417.jpg
-1
processing file %s 4418.jpg
1
processing file %s 4419.jpg
-1
processing file %s 442.jpg
-1
processing file %s 4420.jpg
-1
processing file %s 4421.jpg
-1
processing file %s 4422.jpg
-1
processing file %s 4423.jpg
1
processing file %s 4424.jpg
-1
processing file %s 4425.jpg
1
processing file %s 4426.jpg
-1
processing file %s 4427.jpg
1
processing file %s 4428.jpg
1
processing file %s 4429.jpg
-1
processing file %s 443.jpg
-1
processing file %s 4430.jpg
-1
processing file %s 4431.jpg
1
processing file %s 4432.jpg
-1
processing file %s 4433.jpg
-1
processing file %s 4434.jpg
-1
processing file %s 4435.jpg
-1
processing file %s 4436.jpg
1
processing file %s 4437.jpg
-1
processing file %s 4438.jpg
-1
processing file %s

1
processing file %s 4653.jpg
-1
processing file %s 4654.jpg
1
processing file %s 4655.jpg
1
processing file %s 4656.jpg
-1
processing file %s 4657.jpg
-1
processing file %s 4658.jpg
-1
processing file %s 4659.jpg
-1
processing file %s 466.jpg
-1
processing file %s 4660.jpg
-1
processing file %s 4661.jpg
-1
processing file %s 4662.jpg
-1
processing file %s 4663.jpg
-1
processing file %s 4664.jpg
-1
processing file %s 4665.jpg
1
processing file %s 4666.jpg
1
processing file %s 4667.jpg
-1
processing file %s 4668.jpg
1
processing file %s 4669.jpg
1
processing file %s 467.jpg
-1
processing file %s 4670.jpg
-1
processing file %s 4671.jpg
1
processing file %s 4672.jpg
-1
processing file %s 4673.jpg
-1
processing file %s 4674.jpg
-1
processing file %s 4675.jpg
-1
processing file %s 4676.jpg
-1
processing file %s 4677.jpg
-1
processing file %s 4678.jpg
-1
processing file %s 4679.jpg
-1
processing file %s 468.jpg
-1
processing file %s 4680.jpg
-1
processing file %s 4681.jpg
-1
processing file 

-1
processing file %s 49.jpg
-1
processing file %s 490.jpg
-1
processing file %s 4900.jpg
-1
processing file %s 4901.jpg
1
processing file %s 4902.jpg
-1
processing file %s 4903.jpg
1
processing file %s 4904.jpg
-1
processing file %s 4905.jpg
-1
processing file %s 4906.jpg
-1
processing file %s 4907.jpg
-1
processing file %s 4908.jpg
1
processing file %s 4909.jpg
-1
processing file %s 491.jpg
-1
processing file %s 4910.jpg
-1
processing file %s 4911.jpg
-1
processing file %s 4912.jpg
1
processing file %s 4913.jpg
1
processing file %s 4914.jpg
1
processing file %s 4915.jpg
-1
processing file %s 4916.jpg
-1
processing file %s 4917.jpg
-1
processing file %s 4918.jpg
1
processing file %s 4919.jpg
-1
processing file %s 492.jpg
-1
processing file %s 4920.jpg
1
processing file %s 4921.jpg
-1
processing file %s 4922.jpg
-1
processing file %s 4923.jpg
-1
processing file %s 4924.jpg
-1
processing file %s 4925.jpg
1
processing file %s 4926.jpg
-1
processing file %s 4927.jpg
-1
processing file %s 

-1
processing file %s 652.jpg
-1
processing file %s 653.jpg
1
processing file %s 654.jpg
-1
processing file %s 655.jpg
-1
processing file %s 656.jpg
-1
processing file %s 657.jpg
-1
processing file %s 658.jpg
-1
processing file %s 659.jpg
1
processing file %s 66.jpg
-1
processing file %s 660.jpg
-1
processing file %s 661.jpg
-1
processing file %s 662.jpg
-1
processing file %s 663.jpg
-1
processing file %s 664.jpg
-1
processing file %s 665.jpg
1
processing file %s 666.jpg
-1
processing file %s 667.jpg
1
processing file %s 668.jpg
-1
processing file %s 669.jpg
-1
processing file %s 67.jpg
-1
processing file %s 670.jpg
1
processing file %s 671.jpg
-1
processing file %s 672.jpg
-1
processing file %s 673.jpg
-1
processing file %s 674.jpg
1
processing file %s 675.jpg
-1
processing file %s 676.jpg
1
processing file %s 677.jpg
-1
processing file %s 678.jpg
1
processing file %s 679.jpg
-1
processing file %s 68.jpg
1
processing file %s 680.jpg
1
processing file %s 681.jpg
-1
processing file %s 6

1
processing file %s 901.jpg
-1
processing file %s 902.jpg
-1
processing file %s 903.jpg
-1
processing file %s 904.jpg
1
processing file %s 905.jpg
-1
processing file %s 906.jpg
-1
processing file %s 907.jpg
-1
processing file %s 908.jpg
-1
processing file %s 909.jpg
-1
processing file %s 91.jpg
-1
processing file %s 910.jpg
-1
processing file %s 911.jpg
-1
processing file %s 912.jpg
-1
processing file %s 913.jpg
-1
processing file %s 914.jpg
-1
processing file %s 915.jpg
1
processing file %s 916.jpg
-1
processing file %s 917.jpg
-1
processing file %s 918.jpg
-1
processing file %s 919.jpg
-1
processing file %s 92.jpg
1
processing file %s 920.jpg
-1
processing file %s 921.jpg
-1
processing file %s 922.jpg
-1
processing file %s 923.jpg
-1
processing file %s 924.jpg
-1
processing file %s 925.jpg
-1
processing file %s 926.jpg
1
processing file %s 927.jpg
-1
processing file %s 928.jpg
1
processing file %s 929.jpg
-1
processing file %s 93.jpg
-1
processing file %s 930.jpg
1
processing file %

Image pre-processing, using the OpenCV function blobFromImage(). The function scales (default = 1.0), sets size (227, 227 are common for CNN), subtracts mean, swaps channels (OpenCV assumes images are in BGR channel order, while the mean assumes RGB order. If the value is True, R and B channels are swapped). 

In [8]:
with open('../Datasets/celeba/output.csv', 'w', newline='') as outfile:
    writer = csv.writer(outfile, delimiter = ',')
    writer.writerow(['img_name', 'correct_gender', 'predicted_gender'])
    for predicted_label in predicted_labels:
        correct_labels_row = correct_labels[predicted_label[0]]
        writer.writerow([predicted_label[0], correct_labels_row[2], predicted_label[1]])

The outcome of the classification is written into a gender_labels list. 
To determine model accuracy, the classification output is compared with the existing labels; if classifications match, model accuracy is calculated as % of total number of correct classifications.

In [9]:
accurate_score = 0
for predicted_label in predicted_labels:
    correct_labels_row = correct_labels[predicted_label[0]]
    if predicted_label[1] == correct_labels_row[2]:
        accurate_score +=1
model_accuracy = (accurate_score*100)/len(predicted_labels)
print("Model accuracy : " + str(model_accuracy) + "%")
print(len(predicted_labels))

Model accuracy : 53.18%
5000
